In [ ]:
! pip install flask opencv-python
from flask import Flask, request, render_template
import requests
import json
import numpy as np
import cv2

app = Flask(__name__)

# Function to preprocess image (replace this with your actual preprocessing function)
def preprocess_image(img):
    # Resize image to minimum size of 256x256 while maintaining aspect ratio
    min_size = min(img.shape[:2])
    scale_factor = 256 / min_size
    new_size = (int(img.shape[1] * scale_factor), int(img.shape[0] * scale_factor))
    img_resized = cv2.resize(img, new_size)

    # Crop 224x224 from the center
    center_x = new_size[0] // 2
    center_y = new_size[1] // 2
    half_crop = 112
    img_cropped = img_resized[center_y - half_crop:center_y + half_crop, center_x - half_crop:center_x + half_crop]

    # Normalize pixel values
    mean = np.array([0.485, 0.456, 0.406]) * 255
    std = np.array([0.229, 0.224, 0.225]) * 255
    img_normalized = (img_cropped - mean) / std

    # Transpose image from HWC to CHW layout
    img_transposed = np.transpose(img_normalized, (2, 0, 1))

    return img_transposed

def load_image(image_path):
    return cv2.imread(image_path)

# Function to convert image data to flat array
def image_to_flat_array(image_data):
    return image_data.flatten().tolist()

# Function to convert image data to JSON format
def image_to_json(image_data):
    return json.dumps({"inputs": [{"name": "data", "shape": [1, 3, 224, 224], "datatype": "FP32", "data": image_data}]})

# Function to load class labels
def load_class_labels():
    with open('imagenet_classes.txt', 'r') as f:
        class_labels = f.read().splitlines()
    return class_labels

# Function to perform inference
def perform_inference(image):
    # Preprocess image
    image_processed = preprocess_image(image)
    
    # Convert image to flat array and JSON format
    image_flat = image_to_flat_array(image_processed)
    image_json = image_to_json(image_flat)

    # Send request to OpenVINO server
    url = 'https://onnx-nageshrathod7296-dev.apps.sandbox-m3.1530.p1.openshiftapps.com/v2/models/onnx/infer'
    headers = {'Content-Type': 'application/json'}
    try:
        response = requests.post(url, data=image_json, headers=headers)
        if response.status_code == 200:
            # Parse response
            results = json.loads(response.text)

            # Get class labels
            class_labels = load_class_labels()

            # Get the top-1 prediction
            predictions = np.array(response.json()['outputs'][0]['data'])
            top_prediction_idx = np.argmax(predictions)
            top_prediction_label = class_labels[top_prediction_idx]

            return top_prediction_label
    except Exception as e:
        return "Error: {}".format(e)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return render_template('index.html', message='No file part')
        file = request.files['file']
        # If user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return render_template('index.html', message='No selected file')
        if file:
            # Perform inference
            file.save('image.jpg')
            img = load_image('image.jpg')
            result = perform_inference(img)
            return render_template('result.html', prediction=result)
    return render_template('index.html')


if __name__ == "__main__":
  app.run(debug=False, host="0.0.0.0", port=5000)


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.128.35.87:5000
Press CTRL+C to quit
127.0.0.1 - - [20/Jun/2024 09:37:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2024 09:37:19] "POST / HTTP/1.1" 200 -


<Response [403]>


127.0.0.1 - - [20/Jun/2024 09:38:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jun/2024 09:39:01] "POST / HTTP/1.1" 200 -


<Response [200]>
[ 1.87674030e+00  2.81930520e+00  5.75479570e-01  2.23815820e+00
 -5.98615940e-01  3.52208110e+00 -3.20759000e-01 -4.02696460e-01
 -2.92995930e+00 -1.90220420e+00  2.45158310e+00  4.23712670e-01
 -7.61135200e-01 -1.24586990e+00  2.27937270e+00  6.02862660e-01
 -1.77304660e+00  9.91911000e-01 -5.02863470e-01 -2.09796990e-03
 -1.22699500e+00  1.07224120e-01 -1.97514240e+00 -6.46349850e-01
  4.40141000e-01  1.82682940e+00  4.47938700e+00  3.05363920e-01
  3.25384900e+00  2.61048720e+00 -9.13139200e-02 -1.16211150e+00
  2.61889430e+00  2.41442820e+00  3.02561200e+00 -2.63325450e-01
  2.60614040e+00 -3.40217320e-01  3.03662000e+00  3.26887160e-01
 -6.86207350e-01  1.77117900e+00  6.26734740e-03  1.63566150e+00
  1.12717580e-01 -4.30975100e-02 -2.07109170e+00  2.07654760e+00
 -1.86564120e+00  5.10624650e-01  1.00186190e+00  1.18171380e+00
  1.17814480e+00  1.70162560e+00  2.16327400e+00 -1.39416660e+00
 -1.01302860e-01  2.05244620e-01  2.83712700e+00  6.30629900e-01
  2.2403

In [ ]:
! wget https://github.com/onnx/models/raw/main/validated/vision/classification/resnet/model/resnet152-v2-7.onnx?download=true -O /opt/app-root/src/abc.onnx